In [ ]:
import os
import gc
import sys
import math
import time
import tqdm
import random
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm import tqdm
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold,StratifiedKFold

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from transformers import (AutoModel, AutoTokenizer,AutoConfig,
                          get_cosine_schedule_with_warmup)

from transformers import (AutoModelForMaskedLM, 
                            LineByLineTextDataset,
                          DataCollatorForLanguageModeling,
                          Trainer, TrainingArguments)

from colorama import Fore, Back, Style
y_ = Fore.YELLOW
r_ = Fore.RED
g_ = Fore.GREEN
b_ = Fore.BLUE
m_ = Fore.MAGENTA
c_ = Fore.CYAN
sr_ = Style.RESET_ALL

In [ ]:
test_data = pd.read_csv('../input/commonlitreadabilityprize/test.csv')
config = {
    'lr': 2e-5,
    'wd':0.01,
    'batch_size':16,
    'valid_step':20,
    'max_len':300,
    'epochs':5,
    'nfolds':5,
    'seed':42,

    'model_path':'../input/clrp-roberta-base/clrp_roberta_base',
}


def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONASSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(seed=config['seed'])


In [ ]:
class CLRPTestDataset(Dataset):
    def __init__(self,df,tokenizer,max_len=256):
        self.excerpt = df['excerpt'].to_numpy()
        self.max_len = max_len
        self.tokenizer = tokenizer
    
    def __getitem__(self,idx):
        encode = self.tokenizer(self.excerpt[idx],
                                return_tensors='pt',
                                max_length=self.max_len,
                                padding='max_length',
                                truncation=True)
        
        return encode # encode has format {'input_ids': tensor, 'attention_mask'}
    
    def __len__(self):
        return len(self.excerpt)

In [ ]:
class AttentionHead(nn.Module):
    def __init__(self, in_features, hidden_dim):
        super().__init__()
        self.in_features = in_features
        self.middle_features = hidden_dim

        self.W = nn.Linear(in_features, hidden_dim)
        self.V = nn.Linear(hidden_dim, 1)
        self.out_features = hidden_dim

    def forward(self, features):
        att = torch.tanh(self.W(features))
        score = self.V(att)
        attention_weights = torch.softmax(score, dim=1)
        context_vector = attention_weights * features
        context_vector = torch.sum(context_vector, dim=1)

        return context_vector

In [ ]:
class AttentionModel(nn.Module):
    def __init__(self,path):
        super(AttentionModel,self).__init__()
        self.roberta = AutoModel.from_pretrained(path)  
        self.config = AutoConfig.from_pretrained(path)
        self.head = AttentionHead(self.config.hidden_size,self.config.hidden_size) # self attention
        self.mlp = nn.Sequential(
            nn.Dropout(0.1),
            nn.Linear(self.config.hidden_size,128),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(128,1)
        )

    def forward(self,**xb):
        x = self.roberta(**xb)[0] # last_hidden_state 
        x = self.head(x)
        x = self.mlp(x)
        return x
    

class MeanPoolingModel(nn.Module):
    
    def __init__(self, path):
        super(MeanPoolingModel,self).__init__()
        
        self.config = AutoConfig.from_pretrained(path)
        self.roberta = AutoModel.from_pretrained(path)
        self.layer_norm = nn.LayerNorm(self.config.hidden_size)
        self.linear = nn.Linear(self.config.hidden_size, 1)
        
    def forward(self, **xb):
        attention_mask = xb['attention_mask']
        
        outputs = self.roberta(**xb)
        last_hidden_state = outputs[0]
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(last_hidden_state.size()).float()
        sum_embeddings = torch.sum(last_hidden_state * input_mask_expanded, 1)
        sum_mask = input_mask_expanded.sum(1)
        sum_mask = torch.clamp(sum_mask, min=1e-9)
        mean_embeddings = sum_embeddings / sum_mask
        norm_mean_embeddings = self.layer_norm(mean_embeddings)
        logits = self.linear(norm_mean_embeddings)
        
        preds = logits.squeeze(-1).squeeze(-1)
        
        return preds.view(-1).float()

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(config['model_path'])
test_ds = CLRPTestDataset(test_data,tokenizer,config['max_len'])
valid_dl = DataLoader(test_ds,
                    batch_size = 16,
                    shuffle=False,
                    num_workers = 4,
                    pin_memory=True,
                    drop_last=False)

In [ ]:
pretrain_folder = '../input/pytorch-roberta-meanpooling-attention'

In [ ]:
import gc
total_preds = np.zeros((test_data.shape[0], 1))
model_type = 'attention'
for i in range(config['nfolds']):
    
    model_name = f'model{i}'
    model_path = f'{pretrain_folder}/{model_type}/{model_name}/{model_name}.bin'
    model = None
    if model_type == 'attention':
        model = AttentionModel(config['model_path'])
    elif model_type == 'mean':
        model = MeanPoolingModel(config['model_path'])
    model.load_state_dict(torch.load(model_path))
    preds = []
    for valid_enum in valid_dl:
        x_test = {key:val.reshape(val.shape[0],-1) for key,val in valid_enum.items()}
        preds.append(model(**x_test).detach().numpy())
        gc.collect()
    preds = np.concatenate(preds).reshape(test_data.shape[0], 1)
    total_preds += preds
    gc.collect()
final_preds = total_preds/config['nfolds']

In [ ]:
submissions = pd.read_csv('../input/commonlitreadabilityprize/sample_submission.csv')
submissions['target'] = final_preds
submissions.to_csv('submission.csv', index = False)
submissions